In [1]:
!pip install scikit-learn==1.1.1

     |████████████████████████████████| 31.2 MB 30.9 MB/s            
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 0.24.1
    Uninstalling scikit-learn-0.24.1:
      Successfully uninstalled scikit-learn-0.24.1
You should consider upgrading via the '/opt/app-root/bin/python3.8 -m pip install --upgrade pip' command.


In [2]:
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
from pickle import dump, load
import sklearn

print(sklearn.__version__)

1.1.1


In [3]:
df = pd.read_csv("discount_data/arg-synth-data_1.csv")
df.head()

,Unnamed: 0,Name,Stock,Discount
0,0,Bottle,79,0.66
1,1,Bottle,32,0.56
2,2,Bottle,68,0.64
3,3,Bottle,90,0.68
4,4,Bottle,77,0.65


In [4]:
X_y =  df.to_numpy()
names, counts = X_y[:,1], X_y[:,2]  # get X values
counts = counts.astype(int)
names = names.astype(str)
names.dtype

dtype('<U8')

In [5]:
le = preprocessing.LabelEncoder()
names_encoded = le.fit_transform(names)
print(names_encoded[0],
      names_encoded[201],
      names_encoded[401],
      names_encoded[601],
      names_encoded[801])
## TODO: Save labelencoder object for use in processing data.

0 0 4 4 1


In [6]:
X = np.column_stack((names_encoded, counts))
print(X.shape,
      X.dtype)

(2000, 2) int64


In [7]:
# Finding normalised array of X_Train - TODO: save model state.
scaler = preprocessing.StandardScaler().fit(X)
X_std = scaler.transform(X)
print(X_std[:5])

[[-1.41421356 -0.40154761]
 [-1.41421356 -0.60042258]
 [-1.41421356 -0.44809281]
 [-1.41421356 -0.3550024 ]
 [-1.41421356 -0.41001037]]


In [8]:
y = X_y[:,3]  # get y/label values
y = y.astype(float)
print(y.shape,
      y.dtype)

(2000,) float64


In [9]:
X_train, X_test, y_train, y_test = train_test_split(
        X_std, y, test_size=0.4, random_state=42
    )

In [10]:
n_neighbors=5
knn = KNeighborsRegressor(n_neighbors,weights='uniform')
knn.fit(X_train,y_train)

KNeighborsRegressor()

In [11]:
# Find accuracy on Trainset

train_preds = knn.predict(X_train)
mean_squared_error(y_train, train_preds)

5.1066666666666545e-06

In [12]:
# Find accuracy on Testset

test_preds = knn.predict(X_test)
mean_squared_error(y_test, test_preds) 

8.074999999999981e-06

In [13]:
# save the model, the label encoder, the scaler.

dump(knn, open('discount_models/knn-model_1.pkl', 'wb'))
dump(scaler, open('discount_models/scaler_1.pkl', 'wb'))
dump(le, open('discount_models/label-encoder_1.pkl', 'wb'))

In [14]:
# Testing how to load in the model, scaler, encoder.
# and run on new data point(s)

model = load(open('discount_models/knn-model_1.pkl', 'rb'))
sc = load(open('discount_models/scaler_1.pkl', 'rb'))
enc = load(open('discount_models/label-encoder_1.pkl', 'rb'))

In [15]:
## DATA COMES IN - CLASS NAME ONLY

test_datapoint = np.asarray(['Bottle'])
test_datapoint = enc.transform(test_datapoint)

## CLASS NAME ENCODED - NOW WE ADD A STOCK AMOUNT

test_datapoint = np.asarray([[test_datapoint[0], 61]])
print(test_datapoint)
test_datapoint = sc.transform(test_datapoint)

## SCALED, MADE INTO 2D ARRAY - CAN NOW PREDICT.

print(test_datapoint)
print(model.predict(test_datapoint))

[[ 0 61]]
[[-1.41421356 -0.47771249]]
[0.62]
